In [ ]:
import numpy as np                                        #Used for arrays and array operations
import time                                                     #frame rate
import math
import cv2                                                      #Video and Image Processing
import mediapipe as mp                              #hand estimation
import handDetection as hd                         #to detect hand palm and fingers
from comtypes import CLSCTX_ALL
from ctypes import cast, POINTER
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
# pycaw library used for operating system volume control

widthCam, heightCam = 1280, 720       #dimensions of the webcam window

cp = cv2.VideoCapture(0)                         #used for capturing video initialized as cp
cp.set(3, widthCam)                                 #set function is used to set the parameter indecx of the cam along with size
cp.set(4, heightCam)

previous_time = 0
current_time = 0

handdetect = hd.handDetector(detection_confident=0.8)                   #used for detecting the hand in the frame

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()

volume_range = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0, None) # set volume to 100%

minvol = volume_range[0]
maxvol = volume_range[1]

#print(minvol, maxvol)
#minvol = -74.0
#maxvol = 0.0

vol = 0
volper = 0
volbar = 500

while True:                                                    #while loop is for getting the input untill the cam is turned off
    check, frame = cp.read()                          #reading the frame from camera
    frame = handdetect.findhands(frame, draw_landmark=False)
    lmlist = handdetect.gethandlocation(frame, draw_landmark=False)    #to detect the loaction of hand on the capture window
    
    if len(lmlist) != 0:                                    #if condition is to check whether the thumb and index finger are detected or not
        
        #print(lmlist[4], lmlist[8])
        #lmlist[4] = top point of thumb
        #lmlist[8] = top point of first finger
        
        x1,y1 = lmlist[4][1], lmlist[4][2]               #co-ordintes of thumb
        x2,y2 = lmlist[8][1], lmlist[8][2]              #co-ordintes index finger
        cx,cy = (x1+x2) // 2, (y1+y2) // 2            #co-ordinates of the middle point between thumb and index finger
        
        cv2.circle(frame, (x1,y1), 10, (0,0,255), cv2.FILLED)         #to represent the thumb tip with the circle with red color 
        cv2.circle(frame, (x2,y2), 10, (0,0,255), cv2.FILLED)       #to represent the index finger tip with the circle with red color 
        cv2.circle(frame, (cx,cy), 10, (0,0,255), cv2.FILLED)       #to represent the middle point between thumb and index finger with red color
        cv2.line(frame, (x1,y1), (x2,y2), (255,0,0), 4)                    #this represents the line that joins thumb and index finger through which 
                                                                                                #we can modulate the volume
        handrange = math.hypot(x2-x1, y2-y1)             #maximum range of hand
        
        # print(handrange)
        # Hand range 50 - 400
        # Volume range -74.0 - 0.0
        
        vol = np.interp(handrange, [50,400],[minvol,maxvol])           #initializing minimum and maximum volume
        volbar = np.interp(handrange, [50,400],[500,250])
        volper = np.interp(handrange, [50,400],[0,100])
        volume.SetMasterVolumeLevel(vol, None)
        
        if handrange < 50:                            #if  hand range is less than 50 then the circle color turns black (i.e when both fingers meet)
            cv2.circle(frame, (cx,cy), 10, (0,0,0), cv2.FILLED)
            
        cv2.rectangle(frame, (50,250), (85,500),(0,0,0),3)                     
        cv2.rectangle(frame, (50,int(volbar)), (85,500),(0,255,0),cv2.FILLED)       #represent the volume bar 
        cv2.putText(frame, str(int(volper))+"%", (50,550), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 2)    #represent the volume percentage
        
    current_time = time.time()                                  #initializing the current time with time.time() method
    fps = 1 / (current_time - previous_time)            #calculating frames per second based on 1/difference between current and previous time
    previous_time = current_time

    cv2.putText(frame, "FPS: "+str(int(fps)), (60,60), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)        #displaying fps on the caputure window
    cv2.imshow('Hand Gesture Volume Control', frame)

    if cv2.waitKey(1) & 0xFF == ord('s'):     #if we press s on keyboard then the capturing gets stopped sice the while loop gets breaked
        break

cp.release()  #closes the video file or capturing device
cv2.destroyAllWindows()   #finally we destroy all the windows